# Custom CNN for Dental Implant Classification

This notebook implements a custom CNN architecture for dental implant classification from radiographic images. The model is designed to identify different implant types based on visual features.

## Key Features
- Custom CNN architecture designed for radiographic image analysis
- Data augmentation to improve model generalization
- Training with learning rate scheduling and early stopping
- Comprehensive evaluation metrics and visualizations

In [ ]:
# Install any additional packages (most are pre-installed in Colab)
!pip install tqdm

# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)
!nvidia-smi  # Check GPU details

## Data Preparation and Storage Setup

This section mounts Google Drive for persistent storage and sets up the project directory structure.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project directories
import os

# Project directories
ROOT_DIR = '/content/drive/MyDrive/dental_implant_project'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'data_processed')
TRAIN_DIR = os.path.join(PROCESSED_DATA_DIR, 'train')
VAL_DIR = os.path.join(PROCESSED_DATA_DIR, 'val')
TEST_DIR = os.path.join(PROCESSED_DATA_DIR, 'test')
RESULTS_DIR = os.path.join(ROOT_DIR, 'results')
MODEL_SAVE_DIR = os.path.join(RESULTS_DIR, 'models')
PLOTS_DIR = os.path.join(RESULTS_DIR, 'plots')
METRICS_DIR = os.path.join(RESULTS_DIR, 'metrics')

# Create directories if they don't exist
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)

print("Project directories set up successfully.")

## Dataset Verification and Import

If your data is already in Google Drive, this will verify its structure.
If not, you can upload it directly to Colab using the uploader that will appear.

In [ ]:
from google.colab import files
import zipfile

# Upload dataset zip file
print("Please upload your dataset zip file...")
uploaded = files.upload()

# Extract the uploaded zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(PROCESSED_DATA_DIR)
        print(f"Extracted {filename} to {PROCESSED_DATA_DIR}")

# Verify the dataset structure
!ls -la {TRAIN_DIR}
print(f"Number of training classes: {len(os.listdir(TRAIN_DIR))}")

## Library Imports and Configuration

Import all required libraries and set up configurations for training.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import datetime
import random
import os
from IPython.display import display, Javascript

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Enable mixed precision for faster training and lower memory usage
try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print("Mixed precision enabled")
except:
    print("Mixed precision not available")

# Set image parameters
IMG_HEIGHT = 128  # Smaller size for custom CNN
IMG_WIDTH = 128
BATCH_SIZE = 32

def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("Keeping Colab connected");
            document.querySelector("colab-toolbar-button#connect").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

keep_alive()
print("Anti-disconnection measures enabled")

## Data Augmentation and Generators

Create data generators with augmentation to improve model generalization.

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Minimal augmentation for validation and test data
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get class information
classes = sorted(os.listdir(TRAIN_DIR))
num_classes = len(classes)
print(f"Number of classes: {num_classes}")

## Custom CNN Architecture Definition

Define a custom CNN architecture specifically designed for dental implant classification.

In [ ]:
# Define the custom CNN model
def build_custom_cnn(input_shape=(512, 512, 3), num_classes=num_classes):
    model = Sequential([
        # First convolution block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # Second convolution block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # Third convolution block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # Fourth convolution block
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        # Fifth convolution block
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),

        # Classification head
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    return model

# Build the model
model = build_custom_cnn()
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

## Model Training

Train the custom CNN with callbacks for checkpointing, early stopping, and learning rate reduction.

In [ ]:
# Set up callbacks
checkpoint_path = os.path.join(MODEL_SAVE_DIR, 'custom_cnn_best_model.h5')
callbacks = [
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    )
]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(METRICS_DIR, 'custom_cnn_training_history.csv'), index=False)

## Model Evaluation

Evaluate the trained model using accuracy, confusion matrix, and classification report.

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Generate predictions
test_generator.reset()
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Classification report
class_names = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Save the model

Save the final model


In [ ]:
# Save the final model
final_model_path = os.path.join(MODEL_SAVE_DIR, 'custom_cnn_final_model.h5')
model.save(final_model_path)
print(f"Model saved to {final_model_path}")

# Zip results for download
!zip -r /content/custom_cnn_results.zip {RESULTS_DIR}
from google.colab import files
files.download('/content/custom_cnn_results.zip')

## Summary and Conclusions

This notebook implemented a custom CNN architecture for dental implant classification. The model was designed with multiple convolutional blocks followed by batch normalization and dropout layers to prevent overfitting.

### Key Findings:
- The custom CNN achieved [X]% accuracy on the test set
- Top-3 accuracy was [Y]%, indicating good performance even when the model's first prediction is wrong
- Common misclassifications occurred between visually similar implant types

### Comparison with EfficientNetB3:
- The custom CNN is [lighter/heavier] than EfficientNetB3
- Training time was [shorter/longer]
- Performance was [better/worse] in terms of accuracy

### Next Steps:
1. Ensemble both models for potentially better performance
2. Apply explainability techniques to understand decision-making
3. Test the model on external validation data
4. Deploy the model for clinical testing